In [1]:
import numpy as np
import pandas as pd
import requests
from bs4 import BeautifulSoup
from selenium import webdriver
import time
from selenium.webdriver.common.keys import Keys 
import matplotlib.pyplot as plt
import seaborn as sns

In [2]:
#Creating empty data drame
df=pd.DataFrame({'Name':[''], 'Speciality':[''], 'Degree':[''], 'Year_of_experience':[''] , 'Location':[''] , 
'City':[''] ,'dp_score':[''], 'npv':[''], 'consultation_fee':['']})
#Creating list of cities from where data is to be extracted
lis=['Bangalore','Delhi','Mumbai']
#speciality of doctors whose data to be extracted
Speciality = 'Dentist'
#for loop to take city name form lis and each time change url according to city
for i in lis:
    driver = webdriver.Chrome()
    url = f'https://www.practo.com/search/doctors?results_type=doctor&q=%5B%7B%22word%22%3A%22dentist%22%2C%22autocompleted%22%3Atrue%2C%22category%22%3A%22subspeciality%22%7D%5D&city={i}'
    driver.get(url)
    time.sleep(2)
    
    scroll_pause_time = 2 # You can set your own pause time. dont slow too slow that might not able to load more data
    screen_height = driver.execute_script("return window.screen.height;")  # get the screen height of the web
    A = 1

    while True:
        # scroll one screen height each time
        driver.execute_script("window.scrollTo(0, {screen_height}*{A});".format(screen_height=screen_height, A=A))
        A += 1
        time.sleep(scroll_pause_time)
        # update scroll height each time after scrolled, as the scroll height can change after we scrolled the page
        scroll_height = driver.execute_script("return document.body.scrollHeight;")
        # Break the loop when the height we need to scroll to is larger than the total scroll height
        if (screen_height) * A > scroll_height:
            break
    #after scrolling getting the HTML of webpage and extrancting data
    soup = BeautifulSoup(driver.page_source,'lxml')
    postings = soup.find_all('div' , class_= 'u-border-general--bottom')
    for post in postings:
        try:
            link = post.find('div' , class_ = 'listing-doctor-card').find('a').get('href')
            link_full = 'https://www.practo.com'+link
            driver.get(link_full)
            soup2 = BeautifulSoup(driver.page_source,'lxml')

            #extracting name
            try:
                name = soup2.find('h1' , class_ = 'c-profile__title u-bold u-d-inlineblock').text
            except:
                pass       
            #extracting degree
            try:
                Degree = soup2.find('p' , class_ = 'c-profile__details').text
            except:
                pass
            #extracting years of experience
            try:
                Year_of_experience = soup2.find('div' , class_ = 'c-profile__details').find_all('h2')[-1].text
            except:
                pass
            #extracting location
            try:
                Location = soup2.find('h4' , class_ = 'c-profile--clinic__location').text
            except:
                pass
            #extracting dp score
            try:
                dp_score = soup2.find('span' , class_ = 'u-green-text u-bold u-large-font').text.strip()
            except:
                pass
            #extracting npv
            try:
                npv = soup2.find('span' , class_ = 'u-smallest-font u-grey_3-text').text
            except:
                pass
            #extracting consulting fee
            try:
                consultant_fee = soup2.find('span' , class_ = 'u-strike').text.strip()
            except:
                consultant_fee = soup2.find('div' , class_ = 'u-f-right u-large-font u-bold u-valign--middle u-lheight-normal').text.strip()
            #appending all data into dataframe
            df = df.append({'Name':name, 'Speciality':Speciality, 'Degree':Degree, 'Year_of_experience':Year_of_experience , 'Location':Location , 'City':i ,'dp_score':dp_score, 'npv':npv, 'consultation_fee':consultant_fee} , ignore_index = True)
        except:
            pass

C:\Users\dellpc\AppData\Local\Temp\ipykernel_14480\4052851178.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Name':name, 'Speciality':Speciality, 'Degree':Degree, 'Year_of_experience':Year_of_experience , 'Location':Location , 'City':i ,'dp_score':dp_score, 'npv':npv, 'consultation_fee':consultant_fee} , ignore_index = True)
C:\Users\dellpc\AppData\Local\Temp\ipykernel_14480\4052851178.py:75: FutureWarning: The frame.append method is deprecated and will be removed from pandas in a future version. Use pandas.concat instead.
  df = df.append({'Name':name, 'Speciality':Speciality, 'Degree':Degree, 'Year_of_experience':Year_of_experience , 'Location':Location , 'City':i ,'dp_score':dp_score, 'npv':npv, 'consultation_fee':consultant_fee} , ignore_index = True)
C:\Users\dellpc\AppData\Local\Temp\ipykernel_14480\4052851178.py:75: FutureWarning: The frame.append method is deprecated 

In [3]:
df

,Name,Speciality,Degree,Year_of_experience,Location,City,dp_score,npv,consultation_fee
0,,,,,,,,,
1,Dr. Ganesh Shetty,Dentist,"BDS, MICOI (USA)",25 Years Experience Overall,"Kalyan Nagar, Bangalore",Bangalore,98%,(3787 votes),₹500
2,Dr. Jnanesha H.C,Dentist,"BDS, MDS - Orthodontics",21 Years Experience Overall (16 years as spec...,"Jayanagar 4 Block, Bangalore",Bangalore,98%,(258 votes),₹300
3,Dr. Sumanth Shetty,Dentist,"BDS, MDS - Paedodontics And Preventive Dentistry",24 Years Experience Overall,"Koramangala, Bangalore",Bangalore,97%,(4311 votes),₹300
4,Dr. Anjali Shetty,Dentist,"BDS, MDS - Orthodontics",21 Years Experience Overall (16 years as spec...,"Kalyan Nagar, Bangalore",Bangalore,99%,(1528 votes),₹500
...,...,...,...,...,...,...,...,...,...
1465,Dr. Kunal Shet,Dentist,"BDS, MDS - Orthodontics and Dentofacial Orthop...",14 Years Experience Overall (13 years as spec...,"Vileparle East, Mumbai",Mumbai,99%,(1188 votes),₹ 0 fee
1466,Dr. Devang Bhanushali,Dentist,"MDS - Oral & Maxillofacial Surgery, BDS",16 Years Experience Overall (14 years as spec...,"Mahim, Mumbai",Mumbai,98%,(94 votes),₹ 700 fee
1467,Dr. Bhumita Shah,Dentist,"BDS, PhD - Orthodontics & Dentofacial Orthopae...",16 Years Experience Overall (9 years as speci...,"Andheri West, Mumbai",Mumbai,100%,(33 votes),₹ 200 fee
1468,Dr. Snehal Thakkar,Dentist,"MDS - Oral & Maxillofacial Surgery, BDS",19 Years Experience Overall (18 years as spec...,"Andheri West, Mumbai",Mumbai,100%,(42 votes),₹ 1000 fee


In [4]:
df.nunique()

Name                  1449
Speciality               2
Degree                 120
Year_of_experience     229
Location               323
City                     4
dp_score                23
npv                    457
consultation_fee        39
dtype: int64

In [5]:
df['Speciality'].value_counts()

Dentist    1469
              1
Name: Speciality, dtype: int64

In [6]:
df.to_csv('Dentist.csv',index=False)

In [7]:
df2 = pd.concat(
    map(pd.read_csv, ['Dentist.csv','Gynecologist.csv','Cardiologist.csv','Orthopedist.csv','Rheumatologist.csv','chiropractor.csv','bariatric_surgeon.csv','Pulmonologist.csv','Neurosurgeon.csv','Neurologist.csv','Gastroenterologist.csv','dietitian.csv','Psychiatrist.csv','Urologist.csv','Pediatrician.csv','Dermatologist.csv','Physiotherapist.csv','Infertility Specialist.csv','Ophthalmologist.csv']),ignore_index=True)

In [8]:
df2

,Name,Speciality,Degree,Year_of_experience,Location,City,dp_score,npv,consultation_fee
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Dr. Ganesh Shetty,Dentist,"BDS, MICOI (USA)",25 Years Experience Overall,"Kalyan Nagar, Bangalore",Bangalore,98%,(3787 votes),₹500
2,Dr. Jnanesha H.C,Dentist,"BDS, MDS - Orthodontics",21 Years Experience Overall (16 years as spec...,"Jayanagar 4 Block, Bangalore",Bangalore,98%,(258 votes),₹300
3,Dr. Sumanth Shetty,Dentist,"BDS, MDS - Paedodontics And Preventive Dentistry",24 Years Experience Overall,"Koramangala, Bangalore",Bangalore,97%,(4311 votes),₹300
4,Dr. Anjali Shetty,Dentist,"BDS, MDS - Orthodontics",21 Years Experience Overall (16 years as spec...,"Kalyan Nagar, Bangalore",Bangalore,99%,(1528 votes),₹500
...,...,...,...,...,...,...,...,...,...
5420,Dr. Haresh Asnani,Ophthalmologist,"MBBS, MS - Ophthalmology",37 Years Experience Overall (34 years as spec...,"Chembur East, Mumbai",Mumbai,100%,(17 votes),₹ 800 fee
5421,Dr. Nidhi Thaker,Ophthalmologist,"MBBS, DNB - Ophthalmology",13 Years Experience Overall (5 years as speci...,"Goregaon West, Mumbai",Mumbai,100%,(17 votes),₹ 700 fee
5422,Dr. Sunita Meena,Ophthalmologist,"MBBS, MS - Ophthalmology",14 Years Experience Overall (9 years as speci...,"Chembur, Mumbai",Mumbai,100%,(17 votes),₹ 1000 fee
5423,Dr. Prashant M Koranmath,Ophthalmologist,"MS - Ophthalmology, MD - Physician",9 Years Experience Overall (4 years as specia...,"Malad West, Mumbai",Mumbai,100%,(17 votes),₹ 1200 fee


In [9]:
df2['Speciality'].value_counts()

Dentist                      1469
Gynecologist/obstetrician     917
Infertility Specialist        362
Orthopedist                   320
Physiotherapist               299
Dermatologist                 287
Ophthalmologist               269
Pediatrician                  252
Cardiologist                  206
Urologist                     162
Psychiatrist                  150
Neurosurgeon                  128
dietitian/nutritionist        125
Neurologist                   117
Gastroenterologist            114
Pulmonologist                  90
bariatric surgeon              85
Rheumatologist                 47
chiropractor                    7
Name: Speciality, dtype: int64